In [1]:
import pymongo
from pymongo import MongoClient
import json
import jieba
from gensim import corpora, models, similarities
from collections import defaultdict
import jieba
import io
stoplist = []
documents =[]
words=[]
client = MongoClient() #預設連接自己主機上的mongodb
database = client["test"] # SQL: Database Name
collection = database["test"]
jieba.load_userdict("dict.txt")
with io.open('C:/Users/BIG DATA/Desktop/stoplist.txt','r',encoding = 'utf-8') as f:
    for line in f.readlines():
        stoplist.append(line.strip('\n'))
for post in collection.find():
    documents.append(post['title'])

for doc in range(0,len(documents)):
    word=[]
    z=unicode(documents[doc])
    for a in jieba.cut(z):
        if not a in stoplist:
            ab=[a]
            word+=ab
    #print word
    words+=[word]


    
frequency = defaultdict(int)
for text in words:
    for token in text:
         frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
          for text in words]
dic = corpora.Dictionary(texts)

corpus = [dic.doc2bow(text) for text in texts] 
print corpus[2]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus] 
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=84624) #-1
lsi = models.LsiModel(corpus_tfidf, id2word=dic, num_topics=15)  #分幾群
lsiout=lsi.print_topics(15)                                 #分幾群
corpus_lsi = lsi[corpus_tfidf]

Building prefix dict from the default dictionary ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.cache
Loading model cost 0.365 seconds.
Prefix dict has been built succesfully.


[(0, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]


In [2]:
print lsiout[0][1],"\n"   #第1-15群關鍵詞
print lsiout[1][1],"\n"
print lsiout[2][1],"\n"
print lsiout[3][1],"\n"
print lsiout[4][1],"\n"
print lsiout[5][1],"\n"
print lsiout[7][1],"\n"
print lsiout[8][1],"\n"
print lsiout[9][1],"\n"
print lsiout[10][1],"\n"
print lsiout[11][1],"\n"
print lsiout[12][1],"\n"
print lsiout[13][1],"\n"
print lsiout[14][1],"\n"


print dic  #num_features-1

0.468*"月" + 0.267*"3" + 0.239*"2" + 0.222*"1" + 0.220*"年" + 0.168*"4" + 0.157*"5" + 0.146*"營收" + 0.134*"人" + 0.130*"日" 

-0.514*"月" + 0.338*"立委" + 0.207*"台灣" + -0.201*"營收" + 0.199*"不" + 0.196*"蔡英文" + 0.183*"人" + 0.162*"台" + 0.156*"總統" + 0.135*"中" 

0.611*"元" + -0.362*"月" + -0.305*"立委" + 0.267*"去年" + 0.250*"EPS" + 0.168*"每股" + 0.133*"賺" + 0.097*"台股" + -0.093*"營收" + 0.092*"新高" 

-0.652*"立委" + -0.306*"元" + 0.179*"台灣" + -0.147*"去年" + -0.143*"台" + -0.129*"EPS" + -0.121*"中" + 0.121*"3" + 0.120*"大" + 0.112*"年" 

-0.353*"日盤勢" + -0.350*"賽事" + -0.340*"表" + -0.339*"分析" + -0.240*"立委" + -0.193*"MLB" + 0.189*"月" + -0.174*"3" + -0.173*"2" + 0.168*"蔡英文" 

0.299*"日盤勢" + 0.296*"賽事" + 0.294*"表" + 0.285*"分析" + -0.260*"2" + -0.253*"年" + 0.243*"月" + -0.241*"3" + 0.202*"蔡英文" + -0.198*"1" 

0.694*"大" + 0.291*"台灣" + -0.223*"總統" + -0.211*"蔡英文" + -0.186*"不" + -0.180*"朱立倫" + -0.158*"馬" + 0.147*"立委" + -0.121*"台股" + -0.117*"2" 

-0.670*"3" + 0.349*"2" + 0.311*"台灣" + -0.281*"大" + 0.268*"人" + 0.127*"1" + -0.112*"朱立倫"

In [3]:
index = similarities.MatrixSimilarity(lsi[corpus]) #關鍵字收尋 看是分到第幾群
query = "NBA"
query_bow = dic.doc2bow(list(jieba.cut(query)))
#印出這文章出現哪些訓練好的id
print query_bow
#[(10, 1), (14, 1)]
query_lsi = lsi[query_bow]
#印出和哪個主題類似
print query_lsi

[(1036, 1)]
[(0, 0.034599282742428149), (1, 0.02035126054093956), (2, 0.019844472082962829), (3, 0.058722093072198119), (4, -0.10908450879672653), (5, 0.026611164291909409), (6, -0.0033685360741610823), (7, -0.010040871703680312), (8, -0.031000979978043443), (9, 0.0035525560071409177), (10, 0.0061946777784927581), (11, -0.016882448030013833), (12, 0.010083943737846511), (13, 0.0035229980302851692), (14, 0.0060029301801615212)]


In [4]:
index = similarities.MatrixSimilarity(lsi[corpus])  #關鍵字收尋 看是分到第幾群
query = "MLB"
query_bow = dic.doc2bow(list(jieba.cut(query)))
#印出這文章出現哪些訓練好的id
print query_bow
#[(10, 1), (14, 1)]
query_lsi = lsi[query_bow]
#印出和哪個主題類似
print query_lsi

[(36, 1)]
[(0, 0.019738513434540937), (1, 0.010711210945355662), (2, 0.011906105741372668), (3, 0.067586349773543192), (4, -0.19254688732251232), (5, 0.15559639073917214), (6, 0.027944584906192546), (7, 0.022558971462485974), (8, 0.0092894394650263334), (9, -0.0060192970330498344), (10, 0.0016239497890374224), (11, -0.015110584076753763), (12, 0.0023620916577036274), (13, 0.00079596019002836237), (14, -0.0073962441459338706)]
